In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from functools import partial
import holidays

## Define some functions

In [2]:
from datetime import timedelta
from datetime import datetime
us_holidays = holidays.UnitedStates()

def purchase_date(start, end, freq_per_year, data):
    """
    Given the start, end and number of trading times in a year. Return a list of trading days.
    
    start (datetime)
    
    end (datetime)
    
    freq_per_year (int)
    
    data (pd.Dataframe): history price information
    
    Returns:
        a list of datetime.
    """
    increment = timedelta(days = int(365 / freq_per_year))
    results = []
    current = start
    
    # collect all the available days in data and convert them to python datetime object
    date_in_record = {data.iloc[i]['Date'].to_pydatetime() for i in range(data.shape[0]) }
    
    while current < end:
        # skip days that are not in data
        while not(current in date_in_record):
            current = current + timedelta(days=1)
            
        results.append(current)
        
        current = current + increment
        
    return results


def calculate_trading_amount(data, trade_days, money_per_trade, price_type):
    """
    Calculate the number of holdings purchased in the given trade_days.
    
    data (pd.dataframe): market price history.
    
    trade_days (list of datetime): a list of dates to do trading.
    
    money_per_trade (float): money spent in each trade.
    
    price_type (str): trade base on what type of price. 'Open', 'High', 'Low', 'Close'
    
    Returns:
        a list of tuples with trade_date and the correponding trade amount.
    """
    results = []
    
    for date in trade_days:
        trade_price = data[data['Date'] == date][price_type].values[0]
        num_holdings = money_per_trade / trade_price
        results.append((date, trade_price, num_holdings))
        
    return results
        
    
def add_dividend(trade_info, dividend_data, data, price_type, last_date):
    """
    Add reinvested dividend to the holding
    
    trade_info (tuple): trade_date, trade_price, trade_amount
    
    dividend (pd.DataFrame): Date, Payout Amount
    
    data (pd.DataFrame): stock price history
    
    price_type (str): what kind of price to reinvest. 'Open', 'High', 'Low', 'Close'.
    
    last_date (datetime): the last day to consider dividend.
    
    Returns:
        accumulated holding, accumulated dividend
    """
    trade_dates = [x[0] for x in trade_info]
    min_trade_date = min(trade_dates)
    
    # key: date, value: (trade_price, trade_amount)
    dict_trade_info = {date: (trade_price, trade_amount) for date, trade_price, trade_amount in trade_info}
    
    dividend_dates = [x for x in list(dividend_data['Date']) if (x <= last_date) and (x >= min_trade_date)]
    
    # key: dividend_date, value: (payout, price)
    dict_dividend = {}
    
    for date in dividend_dates:
        price = data[data['Date']==date][price_type].values[0]
        payout = dividend_data[dividend_data['Date']==date]['Payout Amount'].values[0]
        dict_dividend[date] = (payout, price)
    
    # dividend dates and trade dates
    whole_dates = sorted(trade_dates + dividend_dates)
    
    acc_holdings = 0
    acc_reinvested = 0
    
    for date in whole_dates:
        if date in trade_dates:
            acc_holdings += dict_trade_info[date][1]
        elif date in dividend_dates:
            payout, price = dict_dividend[date]
            reinvested_holdings = acc_holdings * payout / price
            acc_reinvested += reinvested_holdings
            acc_holdings += reinvested_holdings
    
    return acc_holdings, acc_reinvested   

## Load ETF Price Data

In [3]:
# Load data and add weekday
data_path = '../data/vti.csv'

data = pd.read_csv(data_path)
data.head()

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['Weekday'] = data['Date'].dt.dayofweek # 0 is Mon, 6 is Sun.
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day


min_time = min(data['Date'])
max_time = max(data['Date'])

print('min_time: ', min_time)
print('max_time: ', max_time)

min_time:  2001-06-15 00:00:00
max_time:  2020-05-08 00:00:00


## Load Dividend Payment Data

In [4]:
# load dividend
dividend_path = '../data/vti_dividend.csv'
dividend = pd.read_csv(dividend_path)


dividend['Date'] = pd.to_datetime(dividend['Ex-Dividend Date'], format='%Y-%m-%d')
dividend['Weekday'] = dividend['Date'].dt.dayofweek
dividend = dividend.drop(['Ex-Dividend Date'], axis=1)
dividend = dividend.drop(['Change'], axis=1)
dividend.head()

,Type,Payout Amount,Date,Weekday
0,Dividend,$0.61,2020-03-26,3
1,Dividend,$0.89,2019-12-24,1
2,Dividend,$0.70,2019-09-16,0
3,Dividend,$0.55,2019-06-17,0
4,Dividend,$0.77,2019-03-25,0


In [5]:
# check that no dividend is issued on weekend
assert len(dividend[(dividend.Weekday==5) & (dividend.Weekday==6)]) == 0
# drop weekday
dividend = dividend.drop(['Weekday'], axis=1)
# drop the split event
dividend = dividend[dividend['Type'] == 'Dividend']

# change payout amount from string to float
dividend['Payout Amount'] = dividend['Payout Amount'].str.replace('$', '').astype(float)

In [6]:
dividend.head()

,Type,Payout Amount,Date
0,Dividend,0.61,2020-03-26
1,Dividend,0.89,2019-12-24
2,Dividend,0.70,2019-09-16
3,Dividend,0.55,2019-06-17
4,Dividend,0.77,2019-03-25


## Calculate Current Asset Value Taking into Account Value Change and Dividend Reinvestment

In [7]:
start = datetime(2010, 1, 1)
end = datetime(2010, 6, 30)

last_date = datetime(2020, 5, 1)

freq_per_year = 1
budget_each_year = 10000
price_type = 'Open'
money_per_trade = budget_each_year / freq_per_year

trade_days = purchase_date(start, end, freq_per_year, data)
trade_info = calculate_trading_amount(data, trade_days, money_per_trade, price_type)
acc_holdings = add_dividend(trade_info, dividend, data, 'Open', last_date)

## Calculate Average APR

In [8]:
tax_rate = 0.15 # long-term capital gain

last_date_price = data[data['Date'] == last_date]['Open'].values[0]
total_value = acc_holdings[0] * last_date_price
premium = len(trade_days) * money_per_trade
print('Premium: ', premium)

# value after tax
gain = total_value - premium
tax = gain * tax_rate
final = total_value - tax
print('value after tax: ', final)

# time duration
period = last_date - start
n_years = period.days / 365
apr = np.power(total_value / premium, 1/ n_years)
apr_tax = np.power(final / premium, 1 / n_years)

print('avg apr before tax: ', apr)
print('avg apr after tax: ', apr_tax)

Premium:  10000.0
value after tax:  27609.874399009637
avg apr before tax:  1.1146787071593678
avg apr after tax:  1.1032363702030437
